## Introduction

The deliverable for this Take Home is this python notebook (`.ipynb` format), updated with your code.

It consists of 3 parts. If you're not able to complete certain aspects of part 1 or 2 with code, simply describe the way you would solve it.

Import the following libraries to get started, installing them on your computer first if needed.

Reach out if you have any queries, and have fun!

In [ ]:
!pip install delta_sharing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 101.6 MB/s eta 0:00:00


## Import Packages

In [ ]:
import requests
import json
import pandas as pd
import delta_sharing
from io import BytesIO
from datetime import datetime

## Explore config file

In [ ]:
profile_file = "config_interview.share"
# Initiate the class
client = delta_sharing.SharingClient(profile_file)
print("List all attributes and methods in class client:", dir(client))
# check list_shares() method
for share in client.list_shares():
    print("Share:", share.name)
# check list_all_tables() method
for table in client.list_all_tables():
    print("Table", table)
    print("Table name:", table.name)
    print("Table schema:", table.schema)
    print("______________________________")

List all attributes and methods in class client: ['_SharingClient__list_all_tables_in_share', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_profile', '_rest_client', 'list_all_tables', 'list_schemas', 'list_shares', 'list_tables']
Share: interview
Table Table(name='unybiz_forms_forms', share='interview', schema='demo')
Table name: unybiz_forms_forms
Table schema: demo
______________________________
Table Table(name='novadetrack_publicprojects', share='interview', schema='demo')
Table name: novadetrack_publicprojects
Table schema: demo
______________________________


## Part 1
Hello, fellow Data Engineer at ChopChop Construction!

Data Provider Nova has stored our data on Databricks, and they've given us the following instructions.

> Hi! Here's documentation on how to retrieve the data you asked for https://github.com/delta-io/delta-sharing/blob/main/PROTOCOL.md#api-response-format-in-delta. The credentials you need are in the config file sent. Our schema name is `demo` and the table names are `unybiz_forms_forms` and `novadetrack_publicprojects`.


Your coworker has called the API and gotten a response. However, he needs help parsing the parquet file links to retrieve the tables in a pandas dataframe format. Write some code that does this and display both tables.

He also removed the Bearer token so you will have to add that back in as well.

## Read parquet table via API

In [ ]:
token="YR9jlOyIg07I8ZPoF6V0kNo0lcN0zWqtNwwFOeaaYqIgY9qDwnhGP6kN61VrLp_W" #As best practice, token should be saved in KeyVault and not hard-coded here
tables = ["unybiz_forms_forms", "novadetrack_publicprojects"]
for table in tables:
    print(f"Processing table: {table}")
    url = f"https://southeastasia.azuredatabricks.net/api/2.0/delta-sharing/metastores/0f7c5957-12d6-4931-a13f-6e65314456b6/shares/interview/schemas/demo/tables/{table}/query"

    # headers is refered to Read Data from a Table Section in https://github.com/delta-io/delta-sharing/blob/main/PROTOCOL.md#list-schemas-in-a-share
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json; charset=utf-8", #optional
        "delta-sharing-capabilities": "responseformat=parquet" #optional
    }
    # payload is refered to Read Data from a Table Section in https://github.com/delta-io/delta-sharing/blob/main/PROTOCOL.md#example-for-snapshot-query
    payload = {
        "predicateHints": [],
        "limitHint": 0,
        "jsonPredicateHints": ""
    }
    # query_params is not required according to the above documentation
    query_params = {"": ""}
    response = requests.post(url, json=payload, headers=headers)
    #print("Response status_code:", response.status_code)
    # Remark: response.json() cannot be used because response is not valid JSON, so response.text is used instead
    print("Response text:", response.text)

    if response.status_code == 200:
        try:
            # Read the response line by line (strip is used to remove any leading whitespace in text and split each line which is separated by \n)
            list_parquet_urls = []
            for line in response.text.strip().split("\n"):
                # load each line to json format for further usage
                record = json.loads(line)
                if "metaData" in record:
                    print("Metadata (Schema of table)", record["metaData"])
                if "file" in record:
                    # append url to list_parquet_urls
                    list_parquet_urls.append(record["file"]["url"])

            if len(list_parquet_urls)==0:
                print(f"There is no parquet files in the table {table}.")
            else:
                # Use pandas read_parquet to load each parquet file into a pandas DataFrame and append it to all_tables
                all_tables = []
                for parquet_url in list_parquet_urls:
                    all_tables.append(pd.read_parquet(parquet_url))
                # Concatinate all tables
                pd_table = pd.concat(all_tables, ignore_index=True)
                print(f"Number of rows in {table} table:", pd_table.shape[0])
                display(pd_table)
        except ValueError as e:
            print(f"Error when processing table {table}: {e}")
    else:
        print(f"Failed to query table {table} -- Response status_code: {response.status_code}")

Processing table: unybiz_forms_forms
Response text: {"protocol":{"minReaderVersion":1}}
{"metaData":{"id":"425438de-09d8-4e26-9e13-a3a585c480bd","format":{"provider":"parquet"},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"id\",\"type\":\"string\",\"nullable\":false,\"metadata\":{}},{\"name\":\"date\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}},{\"name\":\"geo\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"linkedid\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"linkedtable\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"name\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"owner\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"projectid\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"status\",\"type\":\"integer\",\"nullable\":true,\"metadata\":{}},{\"name\":\"history\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"tem

,id,date,geo,linkedid,linkedtable,name,owner,projectid,status,history,templateid,value,module,feature
0,206640_00097D297556E42E98B7558677AFF2,1742390671123,,206640_2593735AD0B542774C9EF6D1715D33:F27,Forms.forms,6,206640_Jiaqi Chien,206640_14,0,,206640_C697FD5A818978DA8A91D75666D10D,"{""F14"":""AET-182595"",""F15"":""form id"",""F16"":""mat...",None,None
1,206640_00AE780CADA26A315E33333BF2C304,1704690566066,,206640_765E1E07690F3AA190EE7D46A14EC4,novadetrack.units,TEC-B12AB,206640_Jiaqi Chien,206640_14,2,"[{""name"":""Requested by PC"",""staff"":""Robert"",""d...",206640_4C9BAB99CD7D9BFBD16645F21F0162,"{""F2"":""1AF76A1EEBE635677ACA718F964525"",""F70"":""...",Safety,Safety Form
2,206640_00B41D5E963B8369C036C0AF408BEF,1742451765478,,206640_04D9D1EAC517053A10CE4110D7EF6C,novadetrack.inspections,CHR-002,206640_Jiaqi Chien,206640_3,1,"[{""name"":""Work In Progress"",""staff"":""Jiaqi Chi...",206640_DB1B73F4E6D670347C46354B249DA9,"{""insp_schedule_date"":1742451765508,""insp_assi...",None,None
3,206640_00F17EA056B781AE2F9F10387D3178,1703566363695,,206640_75C4D3669A535A84802EA3B1EEB14C,novadesafety.ptws,TEH-dupA010101-002,206640_Database Admin,206640_4HXD9I,100,"[{""name"":""Submitted"",""staff"":""Database Admin"",...",206640_111E7726A1E2E9F6712E72A3940C48,"{""F70"":""TEH-dupA010101-002"",""F6"":""Database Adm...",Safety,Permit to Work
4,206640_00FD19ECE8B9CC05229F6174E0A5B2,1715928870640,,206640_287411,novaderesources.people,17,206640_Database Admin,,1,"[{""name"":""Submitted"",""staff"":""Database Admin"",...",206640_84CBF819A8DA3F2F16752AAF82EBD9,"{""F2"":1715928870640,""F3"":""Axel"",""F4"":""Test Mai...",None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12341,206640_583f0dbc6a5fbef7,1742452496739,,206640_bc9a5d47247cdd0,novadetrack.requests,TEH-CONS-043,206640_Noster,206640_4HXD9I,100,"[{""name"":""Submitted (QP)"",""staff"":""Noster"",""da...",206640_6F5AE576902410FB0511656D9A9FEB,{},None,None
12342,206640_C40AAA97C1D7EB92F6573715AAE65C,1743513556657,"1.4450688,103.8286848",206640_27330C836BB27E3C33E276C03B4D12,novadetrack.inspections,28,206640_Jiaqi Chien,206640_4HXD9I,0,,206640_0C518AD7AED009D38DAFAF5906E22F,{},None,None
12343,206640_8BC84057206305CCBEFE316BD31EF4,1743513556633,"1.4450688,103.8286848",206640_27330C836BB27E3C33E276C03B4D12,novadetrack.inspections,TEH-CONS-018,206640_Jiaqi Chien,206640_4HXD9I,1,"[{""name"":""Work In Progress"",""staff"":""Jiaqi Chi...",206640_0D0D524A96DFF7E9E354BAC616BC80,"{""insp_schedule_date"":1743513556678,""insp_assi...",None,None
12344,206640_be60bbe6edae5991,1743522100903,"14.583532,121.177559",206640_765E1E07690F3AA190EE7D46A14EC4,novadetrack.units,TEC-B12AB,206640_Ludwig Facial,206640_14,2,"[{""name"":""Sent to Rejected"",""staff"":""Ludwig Fa...",206640_D182E35C725B5AE96C728CE917FFD3,"{""F9"":1743436800000,""F15"":""0"",""F10"":""1""}",Safety,Safety Form


Processing table: novadetrack_publicprojects
Response text: {"protocol":{"minReaderVersion":1}}
{"metaData":{"id":"bc20f44b-24c6-48ac-9656-3eec5546723f","format":{"provider":"parquet"},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"id\",\"type\":\"string\",\"nullable\":false,\"metadata\":{}},{\"name\":\"appids\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"code\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"name\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"address\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"phase\",\"type\":\"integer\",\"nullable\":true,\"metadata\":{}},{\"name\":\"siteid\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"dbadmin\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"options\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"custom\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":

,id,appids,code,name,address,phase,siteid,dbadmin,options,custom,templateids,ptwtemplateids,wptwtemplateids,safetyFtemplateids,meetingtemplateids,pctemplateids,icdtemplateids,externalkey
0,206640_6,novadefm,MAG,FM Demo,,0,206640_6,pa_presales@novade.net,"{""enableNCR"":""0"",""enablePTW"":""0"",""enableSafety...",,,,,,,,,
1,206640_8,novadefm|novadetrack,SFD,Global Cleaning,,0,206640_ED8E2,pa_presales@novade.net,"{""zone1"":""Country"",""zone2"":""City"",""zone3"":""Gri...",,,,,,,,,
2,206640_16,novadefm,FT,Luxury Resort,"33 33/27 ถนน ศรีสุนทร Tambon Choeng Thale, Tha...",0,206640_6V2UT9,pa_presales@novade.net,"{""projNCRAfterFolderId"":"""",""projectcenter"":{""t...",,,,,,,,,
3,206640_17,novadeactivity|novadetrack,IDR,Renovation,,0,206640_21,pa_presales@novade.net,"{""enableUnitView"":""1"",""zone1"":""Project"",""zone2...",,,,,,,,,
4,206640_27,novadeprojects,K02,Process Reporting,,0,206640_20,pa_presales@novade.net,"{""projNCRAfterFolderId"":""E845597E4E0795DF6C867...",,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,206640_DARLSH,novadetrack|novadeprojects|novadereports|novad...,DC-YTL,Industrial Building Construction - DC,,0,,pa_presales@novade.net,"{""unitLu"":""Q"",""zone1"":""Block / Building"",""zone...",,206640_9B65C67D68EEF93F581099709775EC|206640_D...,,,,,,,
130,206640_WXMO,novadetrack|novadeprojects|novadereports|novad...,RW-YTL,Railway Construction Project,,0,206640_4,pa_presales@novade.net,"{""unitLu"":""Q"",""novadetrack"":{""utabs"":""P|I|F|In...",,206640_F42A25545701DDCCB6CCCDF91A2B37|206640_5...,,,,,,,
131,206640_WV35,novadereports|novadeactivity|novadetime|novade...,WATER,Water Pipelines\r\n& Infrastructure Solutions,,0,206640_E3L7,pa_presales@novade.net,"{""zone1"":""Water Lines"",""zone2"":""KM"",""zone3"":""U...",,206640_340B8AA804306C3E9C2C926F6A091D|206640_F...,,,,,,,
132,206640_F1974,novadetrack,S&C,System & Component,,0,,pa_presales@novade.net,"{""novadetrack"":{""utabs"":""P|I|F|Insp"",""formsCou...",,206640_69E41E4737B385717608BCFC40DE98|206640_D...,,,,,,,


## Part 2
Excited, your manager takes you to the side:

> It wasn't cheap getting our forms digitised... but it's worth it - we finally have all our data in one place!
>
> Analyse this data and give us some suggestions to help the business, the bigger the impact the better!

Analyse the two tables `forms_df` and `projects_df`, covering:
- Business Use Case (Anything you want, but should be business relevant)
- Analysis (e.g. EDA, Data Cleaning, ML methods, BI tools, Visualisations)
- Suggestions (e.g. Actionables, Future Work)

Code has been provided to generate the two tables regardless of whether you completed Part 1.

In [ ]:
# Load the sharing client (save the share in the same directory as this notebook)
profile_file = "config_interview.share"
client = delta_sharing.SharingClient(profile_file)

# Load the tables
form_table_url = f"{profile_file}#interview.demo.unybiz_forms_forms"
project_table_url = f"{profile_file}#interview.demo.novadetrack_publicprojects"

# Convert to pandas DataFrames
forms_df = delta_sharing.load_as_pandas(form_table_url)
projects_df = delta_sharing.load_as_pandas(project_table_url)

In [ ]:
forms_df.describe()

,date,status
count,12346,12346.000000
mean,2023-05-20 00:37:39.312124416,27.778957
min,2018-09-23 07:34:08.367000,-2.000000
25%,2022-08-01 09:46:18.353000192,0.000000
50%,2023-08-11 09:10:02.215000064,1.000000
75%,2024-06-26 09:56:42.238749952,3.000000
max,2025-04-02 02:32:21.080000,300.000000
std,NaN,66.168638


In [ ]:
projects_df.describe()

,phase
count,134.000000
mean,5.216418
std,6.328551
min,0.000000
25%,0.000000
50%,0.000000
75%,12.000000
max,14.000000


## Data Cleaning

In [ ]:
# Convert 'date' to readable datetime if needed
forms_df['date'] = pd.to_datetime(forms_df['date'], unit='ms', errors='coerce')


# Parsing from history infor

def parse_history(row):
    try:
        history = json.loads(row['history']) if isinstance(row['history'], str) else []
        form_id = row['id']
        steps = []
        for i, step in enumerate(history):
            step_name = step.get('name')
            staff = step.get('staff')
            timestamp = step.get('date')
            dt = datetime.fromtimestamp(int(timestamp)/1000) if timestamp else None

            # Duration from previous step
            if i > 0:
                prev_ts = int(history[i-1].get('date', 0))
                duration = (int(timestamp) - prev_ts)/1000/3600
            else:
                duration = None

            steps.append({
                'form_id': form_id,
                'form_status': step_name,
                'staff': staff,
                'form_date': dt,
                'process_duration': duration
            })
        return steps
    except:
        return []


#Parsing from geo infor

def extract_lat_lon(geo):
    try:
        if isinstance(geo, str) and ',' in geo:
            lat, lon = map(float, geo.split(','))
            return pd.Series({'latitude': lat, 'longitude': lon})
    except:
        pass
    return pd.Series({'latitude': None, 'longitude': None})



In [ ]:
history_data = forms_df.apply(parse_history, axis=1).explode().dropna()
history_df = pd.DataFrame(history_data.tolist())
history_df['process_duration'] = history_df['process_duration'].round(2)

In [ ]:
geo_split = forms_df['geo'].apply(extract_lat_lon)

In [ ]:
forms_df_clean = pd.concat([forms_df, history_df], axis=1)
forms_df_clean = pd.concat([forms_df_clean, geo_split], axis=1)
forms_df_clean.drop(['history', 'value', 'owner', 'geo'], axis=1, inplace=True)

In [ ]:
forms_df_clean.head()

,id,date,geo,linkedid,linkedtable,name,projectid,status,templateid,module,feature,form_id,form_status,staff,form_date,process_duration,latitude,longitude
0,206640_00097D297556E42E98B7558677AFF2,2025-03-19 13:24:31.123,,206640_2593735AD0B542774C9EF6D1715D33:F27,Forms.forms,6,206640_14,0.0,206640_C697FD5A818978DA8A91D75666D10D,None,None,206640_00AE780CADA26A315E33333BF2C304,Requested by PC,Robert,2022-02-22 02:56:03.702,NaN,NaN,NaN
1,206640_00AE780CADA26A315E33333BF2C304,2024-01-08 05:09:26.066,,206640_765E1E07690F3AA190EE7D46A14EC4,novadetrack.units,TEC-B12AB,206640_14,2.0,206640_4C9BAB99CD7D9BFBD16645F21F0162,Safety,Safety Form,206640_00AE780CADA26A315E33333BF2C304,Pending Approval,Subcon AUS,2022-02-22 03:04:33.151,0.14,NaN,NaN
2,206640_00B41D5E963B8369C036C0AF408BEF,2025-03-20 06:22:45.478,,206640_04D9D1EAC517053A10CE4110D7EF6C,novadetrack.inspections,CHR-002,206640_3,1.0,206640_DB1B73F4E6D670347C46354B249DA9,None,None,206640_00AE780CADA26A315E33333BF2C304,Duplicated,Jiaqi Chien,2024-01-08 05:09:26.010,16442.08,NaN,NaN
3,206640_00F17EA056B781AE2F9F10387D3178,2023-12-26 04:52:43.695,,206640_75C4D3669A535A84802EA3B1EEB14C,novadesafety.ptws,TEH-dupA010101-002,206640_4HXD9I,100.0,206640_111E7726A1E2E9F6712E72A3940C48,Safety,Permit to Work,206640_00B41D5E963B8369C036C0AF408BEF,Work In Progress,Jiaqi Chien,2025-03-20 06:22:53.655,NaN,NaN,NaN
4,206640_00FD19ECE8B9CC05229F6174E0A5B2,2024-05-17 06:54:30.640,,206640_287411,novaderesources.people,17,,1.0,206640_84CBF819A8DA3F2F16752AAF82EBD9,None,None,206640_00F17EA056B781AE2F9F10387D3178,Submitted,Database Admin,2023-12-26 04:53:50.414,NaN,NaN,NaN


In [ ]:
# Cleaning data for projects_df

# Parse json values

def parse_options(row):
    try:
        opts = json.loads(row['options']) if pd.notnull(row['options']) else {}
    except Exception:
        opts = {}

    output = {
        'enableNCR': opts.get('enableNCR', None),
        'enablePTW': opts.get('enablePTW', None),
        'enableSafety': opts.get('enableSafety', None),
        'zone1': opts.get('zone1', None),
        'zone2': opts.get('zone2', None),
        'zone3': opts.get('zone3', None),
    }

    return pd.Series(output)


MAX_TEMPLATE_COLS = 5

def split_templateids(val):
    if pd.isnull(val):
        return []
    return val.split('|')


In [ ]:
template_split_df = projects_df['templateids'].apply(split_templateids)
parse_projects_df = projects_df.apply(parse_options, axis=1)

In [ ]:
template_split_df = template_split_df.apply(
    lambda x: x + [None]*(MAX_TEMPLATE_COLS - len(x)) if len(x) < MAX_TEMPLATE_COLS else x[:MAX_TEMPLATE_COLS]
)


template_cols = [f'templateid_{i+1}' for i in range(MAX_TEMPLATE_COLS)]
template_df = pd.DataFrame(template_split_df.tolist(), columns=template_cols)


projects_df_clean = pd.concat([projects_df, template_df, parse_projects_df], axis=1)

In [ ]:
projects_df_clean.drop(['options','templateids'], axis=1, inplace=True)

In [ ]:
projects_df_clean.columns

Index(['id', 'appids', 'code', 'name', 'address', 'phase', 'siteid', 'dbadmin',
       'custom', 'ptwtemplateids', 'wptwtemplateids', 'safetyFtemplateids',
       'meetingtemplateids', 'pctemplateids', 'icdtemplateids', 'externalkey',
       'templateid_1', 'templateid_2', 'templateid_3', 'templateid_4',
       'templateid_5', 'enableNCR', 'enablePTW', 'enableSafety', 'zone1',
       'zone2', 'zone3'],
      dtype='object')

In [ ]:
projects_df_clean.to_csv('projects_df.csv', header=True)

In [ ]:
forms_df_clean.to_csv('forms_df.csv', header=True)

## Part 3

Well done! Your manager is ecstatic with your analysis!

She wants to have the code saved in the `Github` repository, and to have the analysis updated with new data daily.

How will you achieve this?

Outline the steps you would take in the `Markdown` cell below, referencing your previous experience with orchestration and version control.

## Github version control

## Solution for updating new data daily

## Option 1: Workfow on Github Actions

name: Daily Update Construction Project

Step 1: define when the workflow will run. i trigger the workflow to run at 11:00pm every day (UTC timezone)
on:
  schedule:
    - cron: '0 23 * * *'

Step 2: define jobs that will be run in this workflow. This workflow contains a single job called "daily_data_update"
jobs:
  daily_data_update:
     Step 2.1: The job will run on ubuntu latest version
    runs-on: ubuntu-latest
     The following steps of the job will be executed in sequence
    steps:
       Step 2.2: Checks-out code repository under $GITHUB_WORKSPACE, so the job can access it: use specific v2 to checkout actions
       uses: actions/checkout@v2

       Step 2.3: Set up Python environment: Python environment is necessary to run this job within Github Actions workflow
       name: Set up Python
        uses: actions/setup-python@v2
        with:
          python-version: '3.10' # use a specific python version

       Step 2.4: Run Data Analysis Notebook: papermill is used to run Jupyter notebook.
       name: Run Data Analysis Notebook
        run: |
          pip install papermill
          papermill interview.ipynb

## Option 2: Workflow on Databricks

Step 1: Generate GitHub Personal Access Token from Github: GitHub settings -> Developer settings -> Personal access tokens -> Generate new token

Step 2: Configure Databricks Repository:

In Databricks Workspace -> Go to Repos -> Create -> select repository provider Github
Copy URL on Github where the code is being saved on Github
Provide GitHub Personal Access Token for authentification
Step 3: Set up Databricks Job:

In Databricks -> Go to Jobs -> Create Job -> Provide job name "Daily Update Construction Project"
Configure Job -> In Task configuration, select Notebook Task -> Select Notebook "name_of_notebook.ipynb"
Configure Cluster for the notebook to execute on: here is an example
Cluster Mode: Standard
Node Type: Standard_DS3_v2
Number of Workers: 4 (choose autoscaling if necessary)
Driver Node: Standard_DS3_v2
Worker Node: Standard_DS3_v2
Termination: Automatically terminate after 45 minutes of inactivity (optional)
Step 4: Schedule Job:

Daily at 23:00 UTC timezone
Email notification on events of failure

## Remark: The difference between Option 1 and Option 2:

In Option 1, GitHub Actions workflow is executed on virtual machines provided by GitHub (e.g. ubuntu). These VMs have predefined resources and environments, and we don't need to manually configure cluster (resurces) like when we run on cloud cluster in Option 2